DATA 1030 Project 1: Population Growth Rate Prediction

Step 0: Import Packages

In [363]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, neighbors, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

# plt.style.use('seaborn-whitegrid')

In [364]:
# Standard imports 

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [365]:
from IPython import __version__ as ipython_version
from pandas import __version__ as pandas_version
from bokeh import __version__ as bokeh_version
print("IPython - %s" % ipython_version)
print("Pandas - %s" % pandas_version)
print("Bokeh - %s" % bokeh_version)

IPython - 6.4.0
Pandas - 0.23.0
Bokeh - 0.12.16


Step 1: Read Data

In [436]:
ts = pd.read_csv('/Users/chuwu/Desktop/project1/time_series_pred.csv') # Kmean+TS model 13 Cluster, Baseline model
ts_20 = pd.read_csv('/Users/chuwu/Desktop/project1/time_series_pred - 20 Clusters.csv') # Kmean+TS model 20 Cluster
ts_0 = pd.read_csv('/Users/chuwu/Desktop/project1/time_series_pred - no Clusters.csv')
ridge = pd.read_csv('/Users/chuwu/Desktop/project1/prediction.csv') # Ridge model, mixed_effect_random_intercept, mixed_effect_random_slope
ts= ts.drop(columns=['year','growth_rate', 'growth_rate_2'])  # left cluster13_2018_pred and country


In [437]:
final_data = pd.read_csv('/Users/chuwu/Desktop/project1/datasets/final_data.csv')

Step 2: seperate predictors to different years

In [438]:
ts_20_2018 = ts_20.loc[ts_20['year'] == 2018]
ts_20_2017 = ts_20.loc[ts_20['year'] == 2017]
ts_20_2016 = ts_20.loc[ts_20['year'] == 2016]

In [439]:
ridge = ridge.rename(columns={"prediction_2018": "ridge_2018","prediction_2017": "ridge_2017","prediction_2016": "ridge_2016",
                              "mdf_2018": "mix_eff1_2018","mdf_2017": "mix_eff1_2017","mdf_2016": "mix_eff1_2016",
                              "mdf_1_2018": "mix_eff2_2018","mdf_1_2017": "mix_eff2_2017","mdf_1_2016": "mix_eff2_2016"})

In [440]:
ts_20_2018 = ts_20_2018.rename(columns={"Baselin": "Baseline_2018","Actual": "Actual_2018","Prediction (20 Clusters": "KMean_TS_C20_2018"})
ts_20_2018
del ts_20_2018['year']

In [441]:
ts_20_2017 = ts_20_2017.rename(columns={"Baselin": "Baseline_2017","Actual": "Actual_2017","Prediction (20 Clusters": "KMean_TS_C20_2017"})
ts_20_2017
del ts_20_2017['year']


In [442]:
ts_20_2016 = ts_20_2016.rename(columns={"Baselin": "Baseline_2016","Actual": "Actual_2016","Prediction (20 Clusters": "KMean_TS_C20_2016"})
ts_20_2016
del ts_20_2016['year']

In [443]:
ts_0_2018 = ts_0.loc[ts_0['year'] == 2018]
ts_0_2017 = ts_0.loc[ts_0['year'] == 2017]
ts_0_2016 = ts_0.loc[ts_0['year'] == 2016]

In [445]:
ts_0_2018 = ts_0_2018.rename(columns={"Prediction": "KMean_TS_C0_2018"})
ts_0_2018
del ts_0_2018['year']
del ts_0_2018['Baseline']
del ts_0_2018['Actual']
ts_0_2017 = ts_0_2017.rename(columns={"Prediction": "KMean_TS_C0_2017"})
ts_0_2017
del ts_0_2017['year']
del ts_0_2017['Baseline']
del ts_0_2017['Actual']

ts_0_2016 = ts_0_2016.rename(columns={"Prediction": "KMean_TS_C0_2016"})
ts_0_2016
del ts_0_2016['year']
del ts_0_2016['Baseline']
del ts_0_2016['Actual']

Step 3: merge cluster_13_2018 to ts_20_2018  # 2018 has 3 models + 1 actual while other years have 2 models +1 actual

In [447]:
ts = ts.rename(columns={"Prediction": "KMean_TS_C13_2018"})

In [448]:
ts_20_2018 = pd.merge(ts_20_2018, ts, how='left', on=['country'])
ts_20_2018

,country,Baseline_2018,Actual_2018,KMean_TS_C20_2018,KMean_TS_C13_2018
0,Bahrain,4.6200,NaN,4.573848,4.511519
1,Oman,4.6700,NaN,4.390391,4.662206
2,Saudi Arabia,2.0300,NaN,2.169900,2.138827
3,Argentina,0.9610,NaN,0.895210,1.049701
4,Colombia,0.8440,NaN,0.732747,0.837635
5,Jamaica,0.3100,NaN,0.158815,0.322771
6,Lebanon,1.2500,NaN,1.285965,1.228272
7,Mauritius,0.0902,NaN,0.082873,0.204496
8,Mexico,1.2600,NaN,1.189237,1.278956
9,Panama,1.5900,NaN,1.581524,1.607560


Step 4: Merge ridge and KMTS  = visual

In [449]:
v = ridge.copy()

In [450]:
v = pd.merge(v, ts_20_2018, how='left', on=['country'])


In [451]:
v = pd.merge(v, ts_20_2017, how='left', on=['country'])
v = pd.merge(v, ts_20_2016, how='left', on=['country'])

In [453]:
v = pd.merge(v, ts_0_2018, how='left', on=['country'])
v = pd.merge(v, ts_0_2017, how='left', on=['country'])
v = pd.merge(v, ts_0_2016, how='left', on=['country'])

In [454]:
v

,country,ridge_2018,ridge_2017,ridge_2016,mix_eff1_2018,mix_eff1_2017,mix_eff1_2016,mix_eff2_2018,mix_eff2_2017,mix_eff2_2016,...,KMean_TS_C13_2018,Baseline_2017,Actual_2017,KMean_TS_C20_2017,Baseline_2016,Actual_2016,KMean_TS_C20_2016,KMean_TS_C0_2018,KMean_TS_C0_2017,KMean_TS_C0_2016
0,Albania,-0.037332,-0.038011,-0.038049,0.452665,0.454758,0.485905,-3.724036,-3.592581,-3.366891,...,-0.078805,-0.1600,-0.0920,-0.081978,-0.2910,-0.1600,-0.215482,-0.105030,-0.172969,-0.140333
1,Algeria,-0.004164,-0.004602,-0.003799,2.013450,2.047216,2.098341,-91.282547,-89.652881,-87.332139,...,1.722964,1.8300,1.7400,1.814589,1.9200,1.8300,1.902839,1.621319,1.734406,1.824398
2,Angola,0.039362,0.039923,0.041337,2.991042,3.003371,3.050261,-53.794195,-52.273445,-51.257438,...,3.171182,3.3700,3.3100,3.317481,3.4300,3.3700,3.381151,3.003414,3.234485,3.317666
3,Antigua and Barbuda,0.140706,0.140078,0.140707,1.344962,1.360464,1.350796,4.078206,4.120917,4.143400,...,1.115619,1.0400,1.0300,1.149581,1.0500,1.0400,1.157263,1.006856,1.051687,1.036786
4,Argentina,0.161871,0.161374,0.163004,0.987963,0.999672,1.035585,-155.497453,-152.165656,-153.382378,...,1.049701,0.9850,0.9610,0.930117,1.0100,0.9850,0.971552,0.936324,0.962544,0.992437
5,Armenia,-0.001031,-0.000622,0.004619,-0.008259,0.001722,0.033397,-4.482575,-4.245604,-4.192622,...,0.195790,0.2690,0.1920,0.303952,0.3690,0.2690,0.397604,0.197296,0.214588,0.301632
6,Australia,-0.076990,-0.076752,-0.070874,1.533527,1.530816,1.517746,-333.718727,-326.532274,-317.300462,...,1.526319,1.5000,1.5900,1.419000,1.4600,1.5000,1.387147,1.490535,1.500561,1.560376
7,Austria,0.056663,0.059511,0.068687,0.801674,0.812428,0.715219,-105.119736,-102.168827,-101.035087,...,0.790462,1.0800,0.8270,0.980494,1.1200,1.0800,0.989448,0.221752,0.802052,1.127170
8,Azerbaijan,0.181681,0.182235,0.190762,1.419142,1.428394,1.549563,-20.337613,-20.166634,-20.227718,...,1.185892,1.1200,1.0700,1.196210,1.1900,1.1200,1.281795,1.474431,1.029671,1.045051
9,Bahrain,0.223923,0.235045,0.247803,2.650274,2.671598,2.698561,-2.711253,-2.335145,-1.989195,...,4.511519,3.8100,4.6200,3.820735,2.6200,3.8100,2.727662,6.025200,4.978473,3.592766


In [455]:
v.to_csv('/Users/chuwu/Desktop/project1/visual_new.csv', index = False)

Step 5: Plot year 2017, 2016 to visualize the performance of each model

#sorted by 2017 actual population growth rate

In [457]:
v_2017_large = pd.read_csv('/Users/chuwu/Desktop/project1/visual_2017_big.csv')
v_2017_large_10 = v_2017_large.head(10)

In [458]:
v_2017_large_10 = v_2017_large.head(10)

In [472]:
from bokeh.models import Legend
from bokeh.plotting import show, figure

# Here is a list of categorical values (or factors)
country = v_2017_large_10.country

# Set the x_range to the list of categories abov
p = figure(x_range=country.values,plot_height=600, plot_width=1024, title="2017 Population Growth Rates by Country (Pred vs Actual)")
p.scatter(country, v_2017_large_10['Baseline_2017'], legend="Baseline", line_width=2,color="red")
p.scatter(country, v_2017_large_10['KMean_TS_C0_2017'], legend="KMean_TS_C0", line_width=2,color="orange",)
p.scatter(country, v_2017_large_10['KMean_TS_C20_2017'], legend="KMean_TS_C20", line_width=2,color="brown",)
p.scatter(country, v_2017_large_10['ridge_2017'], legend="ridge_regression", line_width=2,color="green")
p.scatter(country, v_2017_large_10['mix_eff1_2017'], legend="mixed_effect_model1", line_width=2,color="blue")
p.scatter(country, v_2017_large_10['mix_eff2_2017'], legend="mixed_effect_model2", line_width=2,color="black")
p.vbar(x=country, top=v_2017_large_10['Actual_2017'], width=0.7,color ="wheat",alpha = 0.5,legend = "Actual")

# Set some properties to make the plot look better
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.yaxis.axis_label = "Population Growth Rate %"
p.xaxis.axis_label='Country'
legend = Legend()
p.title.align = 'center'
#p.add_layout(legend, 'right')
p.legend.location = (650, 330)
#p.legend.label_text_font_size = (30)
p.add_layout(legend, 'left')
show(p)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: grass green [renderer: GlyphRenderer(id='e51cd1d7-e711-412e-873b-151c08f90965', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: light blue [renderer: GlyphRenderer(id='0d774bd4-8697-488e-bb40-9b7eb6679e65', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: light blue [renderer: GlyphRenderer(id='4b3a1842-9d23-4109-8ff4-d54b9ade3c97', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: rose [renderer: GlyphRenderer(id='eac1cff0-2add-4559-bdec-6f97e6f9122a', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: light blue [renderer: GlyphRenderer(id='c33be394-50de-4123-bc29-9a514c62553e', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to

In [462]:
v_2016_large = pd.read_csv('/Users/chuwu/Desktop/project1/visual_2016_big.csv')
v_2016_large_10 = v_2016_large.head(10)

In [463]:
from bokeh.models import Legend
from bokeh.plotting import show, figure

# Here is a list of categorical values (or factors)
country = v_2016_large_10.country

# Set the x_range to the list of categories abov
p = figure(x_range=country.values,plot_height=600, plot_width=1024, title="2016 Population Growth Rates by Country (Pred vs Actual)")
p.scatter(country, v_2016_large_10['Baseline_2016'], legend="Baseline", line_width=2,color="red")
p.scatter(country, v_2016_large_10['KMean_TS_C0_2016'], legend="KMean_TS_C0", line_width=2,color="orange",)
p.scatter(country, v_2016_large_10['KMean_TS_C20_2016'], legend="KMean_TS_C20", line_width=2,color="brown",)
p.scatter(country, v_2016_large_10['ridge_2016'], legend="ridge_regression", line_width=2,color="green")
p.scatter(country, v_2016_large_10['mix_eff1_2016'], legend="mixed_effect_model1", line_width=2,color="blue")
p.scatter(country, v_2016_large_10['mix_eff2_2016'], legend="mixed_effect_model2", line_width=2,color="black")
p.vbar(x=country, top=v_2016_large_10['Actual_2016'], width=0.7,color ="wheat",alpha = 0.5,legend = "Actual")

# Set some properties to make the plot look better
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.yaxis.axis_label = "Population Growth Rate %"
p.xaxis.axis_label='Country'
legend = Legend()
p.title.align = 'center'
#p.add_layout(legend, 'right')
p.legend.location = (650, 330)
#p.legend.label_text_font_size = (30)
p.add_layout(legend, 'left')
show(p)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: grass green [renderer: GlyphRenderer(id='e51cd1d7-e711-412e-873b-151c08f90965', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: light blue [renderer: GlyphRenderer(id='0d774bd4-8697-488e-bb40-9b7eb6679e65', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: light blue [renderer: GlyphRenderer(id='4b3a1842-9d23-4109-8ff4-d54b9ade3c97', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: rose [renderer: GlyphRenderer(id='eac1cff0-2add-4559-bdec-6f97e6f9122a', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: light blue [renderer: GlyphRenderer(id='c33be394-50de-4123-bc29-9a514c62553e', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to

Step 6: predict

In [475]:
v = v.rename(columns={"Actual_2018": "Pred_2018_C0"})


In [465]:
v['Pred_2018_C0'] = v['KMean_TS_C0_2018']
v

,country,ridge_2018,ridge_2017,ridge_2016,mix_eff1_2018,mix_eff1_2017,mix_eff1_2016,mix_eff2_2018,mix_eff2_2017,mix_eff2_2016,...,KMean_TS_C13_2018,Baseline_2017,Actual_2017,KMean_TS_C20_2017,Baseline_2016,Actual_2016,KMean_TS_C20_2016,KMean_TS_C0_2018,KMean_TS_C0_2017,KMean_TS_C0_2016
0,Albania,-0.037332,-0.038011,-0.038049,0.452665,0.454758,0.485905,-3.724036,-3.592581,-3.366891,...,-0.078805,-0.1600,-0.0920,-0.081978,-0.2910,-0.1600,-0.215482,-0.105030,-0.172969,-0.140333
1,Algeria,-0.004164,-0.004602,-0.003799,2.013450,2.047216,2.098341,-91.282547,-89.652881,-87.332139,...,1.722964,1.8300,1.7400,1.814589,1.9200,1.8300,1.902839,1.621319,1.734406,1.824398
2,Angola,0.039362,0.039923,0.041337,2.991042,3.003371,3.050261,-53.794195,-52.273445,-51.257438,...,3.171182,3.3700,3.3100,3.317481,3.4300,3.3700,3.381151,3.003414,3.234485,3.317666
3,Antigua and Barbuda,0.140706,0.140078,0.140707,1.344962,1.360464,1.350796,4.078206,4.120917,4.143400,...,1.115619,1.0400,1.0300,1.149581,1.0500,1.0400,1.157263,1.006856,1.051687,1.036786
4,Argentina,0.161871,0.161374,0.163004,0.987963,0.999672,1.035585,-155.497453,-152.165656,-153.382378,...,1.049701,0.9850,0.9610,0.930117,1.0100,0.9850,0.971552,0.936324,0.962544,0.992437
5,Armenia,-0.001031,-0.000622,0.004619,-0.008259,0.001722,0.033397,-4.482575,-4.245604,-4.192622,...,0.195790,0.2690,0.1920,0.303952,0.3690,0.2690,0.397604,0.197296,0.214588,0.301632
6,Australia,-0.076990,-0.076752,-0.070874,1.533527,1.530816,1.517746,-333.718727,-326.532274,-317.300462,...,1.526319,1.5000,1.5900,1.419000,1.4600,1.5000,1.387147,1.490535,1.500561,1.560376
7,Austria,0.056663,0.059511,0.068687,0.801674,0.812428,0.715219,-105.119736,-102.168827,-101.035087,...,0.790462,1.0800,0.8270,0.980494,1.1200,1.0800,0.989448,0.221752,0.802052,1.127170
8,Azerbaijan,0.181681,0.182235,0.190762,1.419142,1.428394,1.549563,-20.337613,-20.166634,-20.227718,...,1.185892,1.1200,1.0700,1.196210,1.1900,1.1200,1.281795,1.474431,1.029671,1.045051
9,Bahrain,0.223923,0.235045,0.247803,2.650274,2.671598,2.698561,-2.711253,-2.335145,-1.989195,...,4.511519,3.8100,4.6200,3.820735,2.6200,3.8100,2.727662,6.025200,4.978473,3.592766


In [467]:
v.to_csv('/Users/chuwu/Desktop/project1/visual_pred.csv', index = False)

In [469]:
v_2018_large = pd.read_csv('/Users/chuwu/Desktop/project1/visual_pred1.csv')
v_2018_large_10 = v_2018_large.head(10)

In [474]:
from bokeh.models import Legend
from bokeh.plotting import show, figure

# Here is a list of categorical values (or factors)
country = v_2018_large_10.country

# Set the x_range to the list of categories abov
p = figure(x_range=country.values,plot_height=600, plot_width=1024, title="2018 Population Growth Rates by Country (Predition)")
p.scatter(country, v_2018_large_10['Baseline_2018'], legend="Baseline", line_width=2,color="red")
p.scatter(country, v_2018_large_10['KMean_TS_C0_2018'], legend="KMean_TS_C0", line_width=2,color="orange",)
p.scatter(country, v_2018_large_10['KMean_TS_C20_2018'], legend="KMean_TS_C20", line_width=2,color="brown",)
p.scatter(country, v_2018_large_10['ridge_2018'], legend="ridge_regression", line_width=2,color="green")
p.scatter(country, v_2018_large_10['mix_eff1_2018'], legend="mixed_effect_model1", line_width=2,color="blue")
p.scatter(country, v_2018_large_10['mix_eff2_2018'], legend="mixed_effect_model2", line_width=2,color="black")
p.vbar(x=country, top=v_2018_large_10['Pred_2018_C0'], width=0.7,color ="wheat",alpha = 0.5,legend = "Prediction_2018")
p.vbar(x=country, top=v_2018_large_10['Actual_2017'], width=0.4,color ="red",alpha = 0.1,legend = "Actual_2017")
# Set some properties to make the plot look better
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.yaxis.axis_label = "Population Growth Rate %"
p.xaxis.axis_label='Country'
legend = Legend()
p.title.align = 'center'
#p.add_layout(legend, 'right')
p.legend.location = (650, 330)
#p.legend.label_text_font_size = (30)
p.add_layout(legend, 'left')
show(p)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: grass green [renderer: GlyphRenderer(id='e51cd1d7-e711-412e-873b-151c08f90965', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: light blue [renderer: GlyphRenderer(id='0d774bd4-8697-488e-bb40-9b7eb6679e65', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: light blue [renderer: GlyphRenderer(id='4b3a1842-9d23-4109-8ff4-d54b9ade3c97', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: rose [renderer: GlyphRenderer(id='eac1cff0-2add-4559-bdec-6f97e6f9122a', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: light blue [renderer: GlyphRenderer(id='c33be394-50de-4123-bc29-9a514c62553e', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to